In [1]:
import pandas as pd
import os

In [2]:
reg_off_df = pd.read_csv('nba_team_stats_data/nba_off.csv')
reg_def_df = pd.read_csv('nba_team_stats_data/nba_def.csv')
reg_adv_df = pd.read_csv('nba_team_stats_data/nba_adv_stats.csv')
post_off_df = pd.read_csv('nba_team_stats_data/nba_post_off.csv')
post_def_df = pd.read_csv('nba_team_stats_data/nba_post_def.csv')
post_adv_df = pd.read_csv('nba_team_stats_data/nba_post_adv_stats.csv')

In [3]:
reg_off_filtered_df = reg_off_df.drop(columns=['Rk'])
reg_off_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [4]:
reg_def_filtered_df = reg_def_df.drop(columns=['Rk'])
reg_def_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [5]:
reg_adv_filtered_df = reg_adv_df.drop(columns=['Rk', 'Unnamed: 17', 'Unnamed: 27', 'Arena', 'Attend.'])
reg_adv_filtered_df.columns

Index(['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%',
       'FT/FGA', 'Unnamed: 22', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1',
       'Attend./G'],
      dtype='object')

In [6]:
post_off_filtered_df = post_off_df.drop(columns=['Rk'])
post_off_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [7]:
post_def_filtered_df = post_def_df.drop(columns=['Rk'])
post_def_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [8]:
post_adv_filtered_df = post_adv_df.drop(columns=['Rk', 'Unnamed: 15', 'Unnamed: 20'])
post_adv_filtered_df.columns

Index(['Team', 'Age', 'W', 'L', 'W/L%', 'PW', 'PL', 'ORtg', 'DRtg', 'NRtg',
       'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
       'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1'],
      dtype='object')

Start Regular Season and Post Season aggregate tables to database

In [9]:
regular_season_df = pd.read_csv('nba_team_stats_data/nba_reg_compiled_stats.csv')
regular_season_df.columns


Index(['Unnamed: 0', 'Team', 'MP', 'Off_FG', 'Off_FGA', 'Off_FG%', 'Off_3P',
       'Off_3PA', 'Off_3P%', 'Off_2P', 'Off_2PA', 'Off_2P%', 'Off_FT',
       'Off_FTA', 'Off_FT%', 'Off_ORB', 'Off_DRB', 'Off_TRB', 'Off_AST',
       'Off_STL', 'Off_BLK', 'Off_TOV', 'Off_PF', 'Off_PTS', 'Def_FG',
       'Def_FGA', 'Def_FG%', 'Def_3P', 'Def_3PA', 'Def_3P%', 'Def_2P',
       'Def_2PA', 'Def_2P%', 'Def_FT', 'Def_FTA', 'Def_FT%', 'Def_ORB',
       'Def_DRB', 'Def_TRB', 'Def_AST', 'Def_STL', 'Def_BLK', 'Def_TOV',
       'Def_PF', 'Def_PTS', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'Off_eFG%',
       'Off_TOV%', 'Off_ORB%', 'Off_FT/FGA', 'Def_eFG%', 'Def_TOV%',
       'Def_DRB%', 'Def_FT/FGA', 'Attend./G'],
      dtype='object')

In [10]:
regular_season_df = regular_season_df.drop(['Unnamed: 0'],axis=1)
regular_season_df.columns = regular_season_df.columns.str.lower()

In [11]:
regular_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg%', 'off_3p', 'off_3pa',
       'off_3p%', 'off_2p', 'off_2pa', 'off_2p%', 'off_ft', 'off_fta',
       'off_ft%', 'off_orb', 'off_drb', 'off_trb', 'off_ast', 'off_stl',
       'off_blk', 'off_tov', 'off_pf', 'off_pts', 'def_fg', 'def_fga',
       'def_fg%', 'def_3p', 'def_3pa', 'def_3p%', 'def_2p', 'def_2pa',
       'def_2p%', 'def_ft', 'def_fta', 'def_ft%', 'def_orb', 'def_drb',
       'def_trb', 'def_ast', 'def_stl', 'def_blk', 'def_tov', 'def_pf',
       'def_pts', 'age', 'w', 'l', 'pw', 'pl', 'mov', 'sos', 'srs', 'ortg',
       'drtg', 'nrtg', 'pace', 'ftr', '3par', 'ts%', 'off_efg%', 'off_tov%',
       'off_orb%', 'off_ft/fga', 'def_efg%', 'def_tov%', 'def_drb%',
       'def_ft/fga', 'attend./g'],
      dtype='object')

In [12]:

regular_season_df = regular_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga', '3par':'three_par',
                                    'attend./g':'attend_g'})
regular_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg_percent', 'off_3p',
       'off_3pa', 'off_3p_percent', 'off_2p', 'off_2pa', 'off_2p_percent',
       'off_ft', 'off_fta', 'off_ft_percent', 'off_orb', 'off_drb', 'off_trb',
       'off_ast', 'off_stl', 'off_blk', 'off_tov', 'off_pf', 'off_pts',
       'def_fg', 'def_fga', 'def_fg_percent', 'def_3p', 'def_3pa',
       'def_3p_percent', 'def_2p', 'def_2pa', 'def_2p_percent', 'def_ft',
       'def_fta', 'def_ft_percent', 'def_orb', 'def_drb', 'def_trb', 'def_ast',
       'def_stl', 'def_blk', 'def_tov', 'def_pf', 'def_pts', 'age', 'w', 'l',
       'pw', 'pl', 'mov', 'sos', 'srs', 'ortg', 'drtg', 'nrtg', 'pace', 'ftr',
       'three_par', 'ts_percent', 'off_efg_percent', 'off_tov_percent',
       'off_orb_percent', 'off_ft_fga', 'def_efg_percent', 'def_tov_percent',
       'def_drb_percent', 'def_ft_fga', 'attend_g'],
      dtype='object')

In [13]:

post_season_df = pd.read_csv('nba_team_stats_data/nba_post_compiled_stats.csv')
post_season_df.columns

Index(['Unnamed: 0', 'Team', 'MP', 'Off_FG', 'Off_FGA', 'Off_FG%', 'Off_3P',
       'Off_3PA', 'Off_3P%', 'Off_2P', 'Off_2PA', 'Off_2P%', 'Off_FT',
       'Off_FTA', 'Off_FT%', 'Off_ORB', 'Off_DRB', 'Off_TRB', 'Off_AST',
       'Off_STL', 'Off_BLK', 'Off_TOV', 'Off_PF', 'Off_PTS', 'Def_FG',
       'Def_FGA', 'Def_FG%', 'Def_3P', 'Def_3PA', 'Def_3P%', 'Def_2P',
       'Def_2PA', 'Def_2P%', 'Def_FT', 'Def_FTA', 'Def_FT%', 'Def_ORB',
       'Def_DRB', 'Def_TRB', 'Def_AST', 'Def_STL', 'Def_BLK', 'Def_TOV',
       'Def_PF', 'Def_PTS', 'Age', 'W', 'L', 'W/L%', 'PW', 'PL', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'Off_eFG%', 'Off_TOV%',
       'Off_ORB%', 'Off_FT/FGA', 'Def_eFG%', 'Def_TOV%', 'Def_DRB%',
       'Def_FT/FGA'],
      dtype='object')

In [14]:
post_season_df = post_season_df.drop(['Unnamed: 0'],axis=1)
post_season_df.columns = post_season_df.columns.str.lower()

In [15]:
{'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga', '3par':'three_par',
                                    'attend./g':'attend_g'})

SyntaxError: unmatched ')' (347157686.py, line 6)

In [16]:

post_season_df = post_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga','3par':'three_par',
                                    'attend./g':'attend_g'})
post_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg_percent', 'off_3p',
       'off_3pa', 'off_3p_percent', 'off_2p', 'off_2pa', 'off_2p_percent',
       'off_ft', 'off_fta', 'off_ft_percent', 'off_orb', 'off_drb', 'off_trb',
       'off_ast', 'off_stl', 'off_blk', 'off_tov', 'off_pf', 'off_pts',
       'def_fg', 'def_fga', 'def_fg_percent', 'def_3p', 'def_3pa',
       'def_3p_percent', 'def_2p', 'def_2pa', 'def_2p_percent', 'def_ft',
       'def_fta', 'def_ft_percent', 'def_orb', 'def_drb', 'def_trb', 'def_ast',
       'def_stl', 'def_blk', 'def_tov', 'def_pf', 'def_pts', 'age', 'w', 'l',
       'w_l_percent', 'pw', 'pl', 'ortg', 'drtg', 'nrtg', 'pace', 'ftr',
       'three_par', 'ts_percent', 'off_efg_percent', 'off_tov_percent',
       'off_orb_percent', 'off_ft_fga', 'def_efg_percent', 'def_tov_percent',
       'def_drb_percent', 'def_ft_fga'],
      dtype='object')

In [17]:
teams_df = post_season_df[['team']]
teams_df = teams_df.rename(columns = {'team':'team_name'})
teams_df.head()

,team_name
0,Golden State Warriors
1,Memphis Grizzlies
2,Denver Nuggets
3,New Orleans Pelicans
4,Minnesota Timberwolves


In [18]:
team_names_series = regular_season_df["team"]

team_abbv = ["MIN", "MEM", "MIL", "CHA", "PHX", "ATL", "UTA", "SAS", "BKN", "DEN", "LAL", "BOS", "CHI", "IND", "GSW", "SAC", "MIA", "PHI", "HOU", "TOR", "NOP", "WAS", "LAC", "DAL", "CLE", "NYK", "POR", "DET", "ORL", "OKC"]
team_abbv_series = pd.Series(team_abbv)

frame = { 'team_name': team_names_series, 'abbreviation': team_abbv_series }

name_abbv = pd.DataFrame(frame)

name_abbv

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


In [19]:
from pprint import pprint
from sqlalchemy import create_engine

In [20]:
rds_connection_string = "postgres:postgres@localhost:5432/group_2_project_4"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [21]:
engine.table_names()

C:\Users\emera\AppData\Local\Temp\ipykernel_11372\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [22]:
name_abbv.to_sql(name='teams', con=engine, if_exists='append', index=False)

30

In [23]:
pd.read_sql_query('select * from teams', con=engine)

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


In [24]:
regular_season_df.to_sql(name='reg_season_stats', con=engine, if_exists='append', index=False)

30

In [25]:
pd.read_sql_query('select * from reg_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga,attend_g
0,Minnesota Timberwolves,241.2,41.6,91.0,0.457,14.8,41.3,0.358,26.8,49.7,...,0.573,0.539,12.4,24.4,0.198,0.535,14.2,74.9,0.227,16028
1,Memphis Grizzlies,241.2,43.5,94.4,0.461,11.5,32.7,0.353,32.0,61.7,...,0.553,0.522,11.2,30.0,0.180,0.523,13.3,77.8,0.195,15775
2,Milwaukee Bucks,240.9,41.8,89.4,0.468,14.1,38.4,0.366,27.8,51.0,...,0.580,0.546,11.9,23.0,0.199,0.536,11.6,78.6,0.165,17453
3,Charlotte Hornets,242.4,42.8,91.4,0.468,13.9,38.2,0.365,28.8,53.3,...,0.572,0.544,11.6,23.3,0.173,0.544,13.1,74.8,0.187,17092
4,Phoenix Suns,240.6,43.7,90.1,0.485,11.6,31.9,0.364,32.1,58.2,...,0.581,0.549,11.6,22.3,0.176,0.510,13.0,77.1,0.195,16175


In [26]:
post_season_df.to_sql(name='post_season_stats', con=engine, if_exists='append', index=False)

16

In [27]:
pd.read_sql_query('select * from post_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,three_par,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga
0,Golden State Warriors,240.0,42.1,85.9,0.490,13.8,36.2,0.382,28.3,49.7,...,0.421,0.599,0.571,13.4,24.2,0.187,0.529,11.6,77.8,0.211
1,Memphis Grizzlies,240.0,39.8,91.3,0.435,13.1,35.8,0.365,26.7,55.5,...,0.392,0.545,0.507,11.9,25.8,0.218,0.530,15.6,77.7,0.211
2,Denver Nuggets,240.0,39.4,82.8,0.476,11.2,31.4,0.357,28.2,51.4,...,0.379,0.586,0.543,14.9,28.0,0.242,0.608,12.2,81.4,0.230
3,New Orleans Pelicans,240.0,39.0,84.3,0.462,9.3,26.3,0.354,29.7,58.0,...,0.312,0.566,0.518,13.0,35.0,0.267,0.575,10.7,80.3,0.193
4,Minnesota Timberwolves,240.0,36.3,82.0,0.443,13.8,35.7,0.388,22.5,46.3,...,0.435,0.579,0.527,15.8,17.1,0.276,0.519,13.6,72.5,0.268


Start Invidual Match Data

In [28]:
df = pd.read_csv('NBA_Data_Reg_Season_2022.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

,team,min,pts,fgm,fga,threepm,threepa,ftm,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,CHI,240,124,44,83,10,21,26,33,16,32,48,22,9,3,23,22,4
1,MIN,240,120,46,91,11,31,17,22,9,23,32,30,7,9,13,23,-4
2,PHI,240,118,46,88,5,25,21,23,10,32,42,25,13,6,11,23,12
3,DET,240,106,38,83,11,34,19,29,15,27,42,26,4,4,20,16,-12
4,BOS,240,139,54,99,18,48,13,13,14,42,56,34,5,2,15,20,29


In [29]:
def winloss(row):
    if row['point_diff'] < 0:
        return 'L'
    return 'W'

In [30]:
df['w_l'] = df.apply(lambda row: winloss(row), axis=1)
df.head()

,team,min,pts,fgm,fga,threepm,threepa,ftm,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff,w_l
0,CHI,240,124,44,83,10,21,26,33,16,32,48,22,9,3,23,22,4,W
1,MIN,240,120,46,91,11,31,17,22,9,23,32,30,7,9,13,23,-4,L
2,PHI,240,118,46,88,5,25,21,23,10,32,42,25,13,6,11,23,12,W
3,DET,240,106,38,83,11,34,19,29,15,27,42,26,4,4,20,16,-12,L
4,BOS,240,139,54,99,18,48,13,13,14,42,56,34,5,2,15,20,29,W


In [31]:
column_to_reorder = df.pop('w_l')
df.insert(1, 'w_l', column_to_reorder)
df

,team,w_l,min,pts,fgm,fga,threepm,threepa,ftm,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,CHI,W,240,124,44,83,10,21,26,33,16,32,48,22,9,3,23,22,4
1,MIN,L,240,120,46,91,11,31,17,22,9,23,32,30,7,9,13,23,-4
2,PHI,W,240,118,46,88,5,25,21,23,10,32,42,25,13,6,11,23,12
3,DET,L,240,106,38,83,11,34,19,29,15,27,42,26,4,4,20,16,-12
4,BOS,W,240,139,54,99,18,48,13,13,14,42,56,34,5,2,15,20,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,IND,L,240,122,42,90,17,47,21,24,8,43,51,29,2,10,17,24,-1
2456,BKN,L,240,104,37,84,17,32,13,23,5,39,44,19,3,9,13,17,-23
2457,MIL,W,240,127,48,105,17,45,14,18,13,41,54,25,8,9,8,19,23
2458,GSW,W,240,121,41,93,14,39,25,30,9,41,50,30,9,2,17,18,7


In [32]:
import numpy as np

In [33]:
df['new'] = df.index
df.head()

,team,w_l,min,pts,fgm,fga,threepm,threepa,ftm,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff,new
0,CHI,W,240,124,44,83,10,21,26,33,16,32,48,22,9,3,23,22,4,0
1,MIN,L,240,120,46,91,11,31,17,22,9,23,32,30,7,9,13,23,-4,1
2,PHI,W,240,118,46,88,5,25,21,23,10,32,42,25,13,6,11,23,12,2
3,DET,L,240,106,38,83,11,34,19,29,15,27,42,26,4,4,20,16,-12,3
4,BOS,W,240,139,54,99,18,48,13,13,14,42,56,34,5,2,15,20,29,4


In [34]:
def type(row):
    if row['new'] % 2 == 0:
        return 'vs'
    return '@'

In [35]:
df['type'] = df.apply(lambda row: type(row), axis=1)
df.head()

,team,w_l,min,pts,fgm,fga,threepm,threepa,ftm,fta,...,dreb,reb,ast,stl,blk,tov,pf,point_diff,new,type
0,CHI,W,240,124,44,83,10,21,26,33,...,32,48,22,9,3,23,22,4,0,vs
1,MIN,L,240,120,46,91,11,31,17,22,...,23,32,30,7,9,13,23,-4,1,@
2,PHI,W,240,118,46,88,5,25,21,23,...,32,42,25,13,6,11,23,12,2,vs
3,DET,L,240,106,38,83,11,34,19,29,...,27,42,26,4,4,20,16,-12,3,@
4,BOS,W,240,139,54,99,18,48,13,13,...,42,56,34,5,2,15,20,29,4,vs


In [36]:
column_to_reorder = df.pop('type')
df.insert(1, 'type', column_to_reorder)
df

,team,type,w_l,min,pts,fgm,fga,threepm,threepa,ftm,...,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff,new
0,CHI,vs,W,240,124,44,83,10,21,26,...,16,32,48,22,9,3,23,22,4,0
1,MIN,@,L,240,120,46,91,11,31,17,...,9,23,32,30,7,9,13,23,-4,1
2,PHI,vs,W,240,118,46,88,5,25,21,...,10,32,42,25,13,6,11,23,12,2
3,DET,@,L,240,106,38,83,11,34,19,...,15,27,42,26,4,4,20,16,-12,3
4,BOS,vs,W,240,139,54,99,18,48,13,...,14,42,56,34,5,2,15,20,29,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,IND,@,L,240,122,42,90,17,47,21,...,8,43,51,29,2,10,17,24,-1,2455
2456,BKN,vs,L,240,104,37,84,17,32,13,...,5,39,44,19,3,9,13,17,-23,2456
2457,MIL,@,W,240,127,48,105,17,45,14,...,13,41,54,25,8,9,8,19,23,2457
2458,GSW,vs,W,240,121,41,93,14,39,25,...,9,41,50,30,9,2,17,18,7,2458


In [37]:
def team(row):
    if row['new'] % 2 == 0:
        return row['new'] - 1
    return row['new'] + 1

In [41]:
df.ix

AttributeError: 'DataFrame' object has no attribute 'ix'

In [38]:
regular_matches_df = pd.read_csv('NBA_Data_Reg_Season_2022_plus.csv')
regular_matches_df = regular_matches_df.drop(['Unnamed: 0'],axis=1)
regular_matches_df.head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,MIN,vs,CHI,L,240,120,46,91,11,31,...,22,9,23,32,30,7,9,13,23,-4
1,CHI,@,MIN,W,240,124,44,83,10,21,...,33,16,32,48,22,9,3,23,22,4
2,PHI,vs,DET,W,240,118,46,88,5,25,...,23,10,32,42,25,13,6,11,23,12
3,DET,@,PHI,L,240,106,38,83,11,34,...,29,15,27,42,26,4,4,20,16,-12
4,MEM,vs,BOS,L,240,110,39,102,15,47,...,27,19,26,45,27,11,6,10,16,-29


In [39]:
regular_matches_df.to_sql(name='reg_season_matches', con=engine, if_exists='append', index=False)

460

In [40]:
pd.read_sql_query('select * from reg_season_matches', con=engine).head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,MIN,vs,CHI,L,240,120,46,91,11,31,...,22,9,23,32,30,7,9,13,23,-4
1,CHI,@,MIN,W,240,124,44,83,10,21,...,33,16,32,48,22,9,3,23,22,4
2,PHI,vs,DET,W,240,118,46,88,5,25,...,23,10,32,42,25,13,6,11,23,12
3,DET,@,PHI,L,240,106,38,83,11,34,...,29,15,27,42,26,4,4,20,16,-12
4,MEM,vs,BOS,L,240,110,39,102,15,47,...,27,19,26,45,27,11,6,10,16,-29


In [42]:
post_matches_df = pd.read_csv('Data/NBA_Data_Playoffs_2022_plus.csv')
post_matches_df = post_matches_df.drop(['Unnamed: 0'],axis=1)
post_matches_df.head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,GSW,vs,BOS,W,240,107,39,86,15,37,...,20,6,36,42,25,15,2,12,17,19
1,BOS,@,GSW,L,240,88,30,80,15,37,...,17,6,37,43,24,5,7,19,18,-19
2,GSW,vs,BOS,L,240,108,39,88,19,45,...,15,12,27,39,24,8,6,14,16,-12
3,BOS,@,GSW,W,240,120,43,85,21,41,...,16,7,32,39,33,7,6,13,13,12
4,MIA,vs,BOS,L,240,96,37,88,6,30,...,24,7,37,44,16,4,5,10,18,-4


In [43]:
post_matches_df.to_sql(name='post_season_matches', con=engine, if_exists='append', index=False)

178

In [44]:
pd.read_sql_query('select * from post_season_matches', con=engine).head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,GSW,vs,BOS,W,240,107,39,86,15,37,...,20,6,36,42,25,15,2,12,17,19
1,BOS,@,GSW,L,240,88,30,80,15,37,...,17,6,37,43,24,5,7,19,18,-19
2,GSW,vs,BOS,L,240,108,39,88,19,45,...,15,12,27,39,24,8,6,14,16,-12
3,BOS,@,GSW,W,240,120,43,85,21,41,...,16,7,32,39,33,7,6,13,13,12
4,MIA,vs,BOS,L,240,96,37,88,6,30,...,24,7,37,44,16,4,5,10,18,-4
